In [2]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


### Select Object Indices of Interest

| Object Indices |[]()|[]()|
|---|---|---|
| __1.__ 002_master_chef_can | __8.__ 009_gelatin_box      | __15.__ 035_power_drill       |
| __2.__ 003_cracker_box     | __9.__ 010_potted_meat_can  | __16.__ 036_wood_block        |
| __3.__ 004_sugar_box       | __10.__ 011_banana          | __17.__ 037_scissors          |
| __4.__ 005_tomato_soup_can | __11.__ 019_pitcher_base    | __18.__ 040_large_marker      |
| __5.__ 006_mustard_bottle  | __12.__ 021_bleach_cleanser | __19.__ 051_large_clamp       |
| __6.__ 007_tuna_fish_can   | __13.__ 024_bowl            | __20.__ 052_extra_large_clamp |
| __7.__ 008_pudding_box     | __14.__ 025_mug             | __21.__ 061_foam_brick        |

In [3]:
from transforms3d.quaternions import quat2mat, mat2quat

def getPoseCNNQuat(data, obj):
    pose_idx = np.where(data['rois'][:,1].flatten()==obj)[0]
    if(len(pose_idx) == 0):
        return None
    else:
        pose_idx = pose_idx[0]
    pose = data['poses'][pose_idx]
    q = pose[:4][[1,2,3,0]]
    q /= np.linalg.norm(q)
    t = pose[4:7]
    return q

## YCB Dataset

In [4]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.image_processing import ImageNormalizer
from object_pose_utils.datasets.inplane_rotation_augmentation import InplaneRotator

from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.IMAGE_CROPPED,
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES,
                ]

dataset = YCBDataset(dataset_root, mode=mode,
                     object_list = list(range(1,22)),
                     output_data = output_format,
                     resample_on_error = False,
                     add_syn_background = False,
                     add_syn_noise = False,
                     use_posecnn_data = True,
                     #preprocessors = [InplaneRotator()],
                     postprocessors = [ImageNormalizer()],
                     image_size = [640, 480], num_points=1000)

model_clouds = {}
for object_id in object_list:
    cloud_filename = '{}/models/{}/points.xyz'.format(dataset_root, dataset.classes[object_id])
    model_clouds[object_id] = np.loadtxt(cloud_filename)

## Pose Estimator

In [5]:
from dense_fusion.network import PoseNet, PoseNetGlobal, PoseRefineNet, PoseNetDropout

df_weights = '/home/bokorn/src/DenseFusion/trained_checkpoints/ycb/pose_model_26_0.012863246640872631.pth'
df_estimator = PoseNet(num_points = 1000, num_obj = 21)
df_estimator.load_state_dict(torch.load(df_weights))
df_estimator.cuda();
df_estimator.eval();

#df_refine_weights = '/home/bokorn/src/DenseFusion/trained_checkpoints/ycb/pose_refine_model_69_0.009449292959118935.pth'
#df_refine_estimator = PoseRefineNet(num_points = 1000, num_obj = 21)
#df_refine_estimator.load_state_dict(torch.load(df_refine_weights))
#df_refine_estimator.cuda();
#df_refine_estimator.eval();

#df_dropout_weights = '/home/bokorn/src/DenseFusion/trained_models/ycb_dropout/pose_model_36_0.019743827587456554.pth'
#df_dropout_estimator = PoseNetDropout(num_points = 1000, num_obj = 21)
#df_dropout_estimator.load_state_dict(torch.load(df_dropout_weights))
#df_dropout_estimator.cuda();
#df_dropout_estimator.eval();

In [14]:
from generic_pose.datasets.ycb_dataset import getYCBSymmeties
from object_pose_utils.utils.pose_processing import symmetricAngularDistance
from object_pose_utils.utils.pose_error import add, adi
from quat_math import quaternion_matrix
from generic_pose.utils.evaluation_utils import evaluateDenseFusion

import pathlib

from tqdm import tqdm_notebook as tqdm

pose_gt = {obj:{} for obj in object_list}
pose_est = {obj:{} for obj in object_list}
sym_angular_error = {obj:{} for obj in object_list}
add_error = {obj:{} for obj in object_list}
add_sym_error = {obj:{} for obj in object_list}

bad_data = []
with torch.no_grad():
    for j, data in enumerate(tqdm(dataset)):
        obj, quat, trans, img, points, choose = data

        if(len(obj) == 0):
            bad_data.append(j)
            continue

        sym_axis, sym_ang = getYCBSymmeties(obj.item())
        trans = to_np(trans)
        
        q_est, t_est, _ = evaluateDenseFusion(df_estimator, img, points, choose, obj)


        mat = quaternion_matrix(quat)
        err_ang = symmetricAngularDistance(torch.Tensor(q_est).unsqueeze(0), 
                                           quat.unsqueeze(0),
                                           sym_axis, sym_ang).item()*180/np.pi
        mat_est = quaternion_matrix(q_est)
        err_add = add(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                      model_clouds[obj.item()])
        err_adi = adi(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                      model_clouds[obj.item()])

        pose_est[obj.item()][j]=(q_est, t_est)
        pose_gt[obj.item()][j]=(to_np(quat), trans)
        sym_angular_error[obj.item()][j]=err_ang
        add_error[obj.item()][j]=err_add
        add_sym_error[obj.item()][j]=err_adi
        
    np.savez('iros_results/dense_fusion.npz', 
             pose_gt=pose_gt,
             pose_est=pose_est,
             sym_angular_error=sym_angular_error,
             add_error=add_error,
             add_sym_error=add_sym_error,
            )

Exception on index 6530: Mask 0049/001052 has less than minimum number of pixels (31 < 50)
Exception on index 6531: Mask 0049/001067 has less than minimum number of pixels (44 < 50)
Exception on index 6556: Mask 0049/001319 has less than minimum number of pixels (47 < 50)
Exception on index 6558: Mask 0049/001340 has less than minimum number of pixels (48 < 50)
Exception on index 6837: Mask 0059/000037 has less than minimum number of pixels (0 < 50)
Exception on index 6842: Mask 0059/000088 has less than minimum number of pixels (0 < 50)
Exception on index 6861: Mask 0059/000162 has less than minimum number of pixels (2 < 50)
Exception on index 6862: Mask 0059/000163 has less than minimum number of pixels (7 < 50)


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Exception on index 12401: Mask 0048/000656 has less than minimum number of pixels (0 < 50)
Exception on index 12412: Mask 0048/000806 has less than minimum number of pixels (0 < 50)
Exception on index 13051: Mask 0054/001882 has less than minimum number of pixels (48 < 50)
Exception on index 13052: Mask 0054/001888 has less than minimum number of pixels (28 < 50)
Exception on index 13054: Mask 0054/001918 has less than minimum number of pixels (0 < 50)
Exception on index 13558: Mask 0057/000830 has less than minimum number of pixels (0 < 50)
Exception on index 13579: Mask 0057/000940 has less than minimum number of pixels (0 < 50)
Exception on index 13580: Mask 0057/000942 has less than minimum number of pixels (0 < 50)
Exception on index 13587: Mask 0057/000965 has less than minimum number of pixels (12 < 50)
Exception on index 13589: Mask 0057/000973 has less than minimum number of pixels (50 < 50)
Exception on index 13595: Mask 0057/000991 has less than minimum number of pixels (12 

In [18]:
#func(img, points, choose, obj)
df_obj_idx = obj - 1
img = img.unsqueeze(0).cuda()
points = points.unsqueeze(0).cuda()
choose = choose.unsqueeze(0).cuda()
df_obj_idx = df_obj_idx.unsqueeze(0).cuda()

In [11]:
(q_est, t_est)
#(quat, trans)
#max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj, use_global_feat=False)
#feat.shape
#lik_est = evaluateFeature(comp_estimator, obj, feat, grid_features)
#comp_estimator(feat_global)
#plt.plot(to_np(lik_est.flatten()))

(array([ 0.9146651 ,  0.17347349,  0.3600927 , -0.06023394], dtype=float32),
 array([0.11527348, 0.06846315, 0.8546898 ], dtype=float32))

In [18]:
print(makeTableEntries(likelihood, val_func=mean_log_clean, bold_func = max, individual = True))
#makeTable(sym_angular_error, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTable(add_error, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTable(add_sym_error, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTableEntries(sym_angular_error_mode, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTableEntries(add_error_mode, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTableEntries(add_sym_error_mode, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')



master chef can
 & \textbf{0.93} \\
cracker box
 & \textbf{2.36} \\
sugar box
 & \textbf{2.05} \\
tomato soup can
 & \textbf{1.42} \\
mustard bottle
 & \textbf{2.22} \\
tuna fish can
 & \textbf{0.08} \\
pudding box
 & \textbf{-1.17} \\
gelatin box
 & \textbf{2.39} \\
potted meat can
 & \textbf{0.63} \\
banana
 & \textbf{0.39} \\
pitcher base
 & \textbf{2.22} \\
bleach cleanser
 & \textbf{1.22} \\
bowl
 & \textbf{-0.38} \\
mug
 & \textbf{-1.28} \\
power drill
 & \textbf{1.75} \\
wood block
 & \textbf{-3.05} \\
scissors
 & \textbf{0.43} \\
large marker
 & \textbf{0.10} \\
large clamp
 & \textbf{-9.64} \\
extra large clamp
 & \textbf{-5.50} \\
foam brick
 & \textbf{-2.42} \\
\hline
All 
 & \textbf{0.07} \\



In [19]:
print(makeTableEntries(likelihood, objs = non_sym_objs, val_func=mean_log_clean, bold_func = max, agg_title = 'Non-Symmetric'))
print(makeTableEntries(likelihood, objs = sym_objs, val_func=mean_log_clean, bold_func = max, agg_title = 'Symmetric'))
print(makeTableEntries(likelihood, val_func=mean_log_clean, bold_func = max))

Non-Symmetric 
 & \textbf{1.10} \\

Symmetric 
 & \textbf{-5.22} \\

All 
 & \textbf{0.07} \\



In [ ]:
def histLikelihood(lik_est, quat, k = 4):
    if(type(lik_est) is torch.Tensor):
        lik_est = to_np(lik_est.flatten())
    tetra_interp.setValues(lik_est)
    lik = tetra_interp.smooth(to_np(quat), k=k).item()
    return lik

In [ ]:
lik_est, q_est, t_est = lik_funcs['hist_conf'](img, points, choose, obj)

tetra_interp.setValues(lik_est)
d, j = tetra_interp.kd_grid.query(to_np(quat), k=20)

print(lik_est[j])
max_idx = np.argmax(lik_est)
q_max = grid_vertices[max_idx]
print(tetra_interp.smooth(to_np(q_max), k=k).item())

In [ ]:
dists = quatAngularDiffBatch(to_np(quat), to_np(grid_vertices))
np.argsort(dists)[max_idx]
dists[max_idx]*180/np.pi

In [ ]:
print(histLikelihood(np.ones(grid_size), quat, k = 3885))

lik_est, q_est, t_est = lik_funcs['hist_reg_global'](img, points, choose, obj)
print(lik_est.shape)
print(histLikelihood(lik_est, quat, k = 3885), lik_est.min(), lik_est.max())

lik_est, q_est, t_est = lik_funcs['hist_conf'](img, points, choose, obj)
print(lik_est.shape)
print(histLikelihood(lik_est, quat, k = 3885), lik_est.min(), lik_est.max())

In [ ]:
def scatterSO3New(vertices, vals = None, q_gt = None,
                   clims = None, 
                   alims = [0.,1.], 
                   s=None,
                   ax = None, cmap = 'jet'):
   
    if(type(cmap) == str):
        cmap = plt.get_cmap(cmap)
    if(ax is None):
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1, projection='3d')

    #c = cmap(vals)
    if(vals is not None):
        if(clims is not None):
            c_min, c_max = clims
        else:
            c_min, c_max = min(vals), max(vals)
            
        a = np.maximum(0,np.minimum(1, (vals - c_min)/(c_max - c_min)))
        c = cmap(a)
        alpha_min, alpha_max = alims
        c[:,3] = (alpha_max-alpha_min)*a + alpha_min
    else:
        a = 1
        c = None

    if(s is None):
        s = a*10
    pts = quats2Point(vertices)
    h = ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=s, c=c)

    if(q_gt is not None):
        pt_gt = quats2Point(q_gt)
        ax.scatter(pt_gt[:,0], pt_gt[:,1], pt_gt[:,2], c='r', marker='x')

    return h


In [ ]:
%matplotlib inline
pylab.rcParams['figure.figsize'] = 20, 12

#idx = 12492 #np.random.randint(len(dataset))
idx = np.random.randint(len(dataset))
data = dataset[idx]
obj, quat, trans, img, points, choose = data
print(obj)
max_q, max_t, feat = evaluateDenseFusion(df_global_estimator, img, points, choose, obj)

imshowTorch(img, normalized=True)

knn_k = 200

max_pt = quats2Point([max_q])
gt_pt = quats2Point([to_np(quat)])

fig = plt.figure()
ax = fig.add_subplot(1,4,1, projection='3d')

print('Uniform: {}'.format(histLikelihood(np.ones(grid_size), quat, k = knn_k)))

#lik_est = evaluateFeature(reg_estimator, obj, feat, grid_features)
key = 'hist_reg_global'
lik_est, q_est, t_est = lik_funcs[key](img, points, choose, obj)
print('{}: {}'.format(key, histLikelihood(lik_est, quat, k = knn_k)))
#lik_est = lik_est > .001
scatterSO3New(to_np(grid_vertices), lik_est, ax=ax, clims = [0,.3], alims = [0,1], s=10)
ax.scatter(max_pt[:,0], max_pt[:,1], max_pt[:,2], c='k', s=100, marker='x')
ax.scatter(gt_pt[:,0], gt_pt[:,1], gt_pt[:,2], c='g',s=100, marker='x')

ax = fig.add_subplot(1,4,2, projection='3d')

key = 'hist_reg_idv_global'
lik_est, q_est, t_est = lik_funcs[key](img, points, choose, obj)
print('{}: {}'.format(key, histLikelihood(lik_est, quat, k = knn_k)))
#lik_est = lik_est > .001
scatterSO3New(to_np(grid_vertices), lik_est, ax=ax, clims = [0,.3], alims = [0,1], s=10)
ax.scatter(max_pt[:,0], max_pt[:,1], max_pt[:,2], c='k', s=100, marker='x')
ax.scatter(gt_pt[:,0], gt_pt[:,1], gt_pt[:,2], c='g', s=100, marker='x')

key = 'hist_comp_global'
ax = fig.add_subplot(1,4,3, projection='3d')
lik_est, q_est, t_est = lik_funcs[key](img, points, choose, obj)
print('{}: {}'.format(key, histLikelihood(lik_est, quat, k = knn_k)))
#lik_est = lik_est > .001
scatterSO3New(to_np(grid_vertices), lik_est, ax=ax, clims = [0,.3], alims = [0,1], s=10)
ax.scatter(max_pt[:,0], max_pt[:,1], max_pt[:,2], c='k', s=100, marker='x')
ax.scatter(gt_pt[:,0], gt_pt[:,1], gt_pt[:,2], c='g', s=100, marker='x')

key = 'hist_conf'
ax = fig.add_subplot(1,4,4, projection='3d')
lik_est, q_est, t_est = lik_funcs[key](img, points, choose, obj)
print('{}: {}'.format(key, histLikelihood(lik_est, quat, k = knn_k)))
#lik_est = lik_est > .001
scatterSO3New(to_np(grid_vertices), lik_est, ax=ax, clims = [0,.3], alims = [0,1], s=10)
ax.scatter(max_pt[:,0], max_pt[:,1], max_pt[:,2], c='k', s=100, marker='x')
ax.scatter(gt_pt[:,0], gt_pt[:,1], gt_pt[:,2], c='g', s=100, marker='x')
plt.show()

In [ ]:
%matplotlib notebook 
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection='3d')

lik_est, q_est, t_est = lik_funcs['hist_conf'](img, points, choose, obj)
#lik_est, q_est, t_est = lik_funcs['hist_reg_idv_global'](img, points, choose, obj)
scatterSO3New(to_np(grid_vertices), lik_est, [to_np(quat)], ax=ax, clims = [0,.3], alims = [0,1], s=1000)
gt_pt = quats2Point([to_np(quat)])
ax.scatter(gt_pt[:,0], gt_pt[:,1], gt_pt[:,2], c='b',s=100, marker='x')
#scatterSO3New(to_np(grid_vertices), dists < np.pi/2., [to_np(quat)], ax=ax, clims = [0,1], alims = [0,.01], s=10)
plt.show()
